In [15]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.brain.internal.models as fbm
import numpy as np

model = fbm.load_model("simple-resnet-cifar10")

def find_unique(dataset_dir,sub_thresh=0.01,subset_files=None):
	try:
		fo.core.dataset.delete_non_persistent_datasets()
	except:
		pass
	dataset = fo.Dataset.from_dir(dataset_dir=dataset_dir,dataset_type=fo.types.ImageDirectory,name='temp1',)
	if subset_files is not None:
		for sample in dataset:
			if sample.filename not in subset_files:
				dataset.delete_samples(sample.id)
	embeddings = dataset.compute_embeddings(model, batch_size=16)
	results = fob.compute_similarity(dataset, embeddings=embeddings, brain_key="img_sim")
	uniques,threshes=[],[]
	for thresh in np.arange(0.01,1,0.01):
		try:
			results.find_duplicates(thresh=thresh)
			print('Unique samples found:'+ str(len(results.unique_ids)) + ' at thresh='+str(thresh))
			uniques.append(len(results.unique_ids))
			threshes.append(thresh)
			if len(uniques)>1:
				if ((uniques[-2]-uniques[-1])/len(dataset))<(sub_thresh*0.5):
					break
		except:
			pass
	arr=np.round(np.diff(np.flip(uniques))/len(dataset),4)>sub_thresh
	print("the values in arr:",arr)
	cont=len(uniques)-1
	found=False
	for j in range(cont,0,-1):
		for i in range(len(arr)):
			if i+j==len(arr)+1:
				break
			if arr[i:i+j].all():
				idx=i
				found=True
				break
		if found:
			cont=j
			break
	try:
		idx
	except NameError:		
		if len(uniques)>1:
			if ((uniques[-2]-uniques[-1])/len(dataset))<(sub_thresh*0.5):
				idx = len(uniques)-1
					
	thresh=np.flip(threshes)[idx]
	results.find_duplicates(thresh=thresh)
	print('\nThreshold fixed at:'+str(thresh)+' at continuity count of '+str(cont))
	print('Total Unique samples collected:', len(results.unique_ids))
	unique_fname=[sample.filename for sample in dataset if sample.id in results.unique_ids]
	dataset.delete()
	return unique_fname


if __name__=='__main__':
	dataset_dir='C://Users//320199927//Desktop//DATA4exp//New folder' # Provide path to image folder
	unique_files=find_unique(dataset_dir,sub_thresh=0.9,subset_files=None)
        

 100% |█████████████████| 332/332 [83.8ms elapsed, 0s remaining, 4.0K samples/s]   
 100% |█████████████████| 332/332 [4.8s elapsed, 0s remaining, 68.6 samples/s]      
Generating index...
Index complete
Computing duplicate samples...
Duplicates computation complete
Unique samples found:146 at thresh=0.01
Computing duplicate samples...
Duplicates computation complete
Unique samples found:146 at thresh=0.02
the values in arr: [False]
Computing duplicate samples...
Duplicates computation complete

Threshold fixed at:0.01 at continuity count of 1
Total Unique samples collected: 146


In [13]:
print(unique_files)

['1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00001.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00022.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00024.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00026.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00028.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00030.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00032.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00035.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00037.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00039.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00041.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00043.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00045.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00047.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00050.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00052.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00054.png', '1.3.6.1.4.1.35190.4.1.20210608_-img-00001-00056.png', '1.3.6.1.